In [ ]:
import mysql.connector
import pandas as pd

# MySQL connection details
mysql_host = 'mysql'
mysql_user = 'student'
mysql_password = 'student'
mysql_database = 'workshop_db'

# Create a connection to the MySQL database
conn = mysql.connector.connect(
    host=mysql_host,
    user=mysql_user,
    password=mysql_password,
    database=mysql_database
)

# UC-1

In [ ]:
# Q1
full_join_query = """SELECT count(*)
FROM beers
WHERE TRUE
;"""
df = pd.read_sql_query(full_join_query, con=conn)
df

In [ ]:
# Q2
full_join_query = """SELECT brew.name as brewery, count(*)
FROM beers
JOIN breweries as brew on brew.id = beers.brewery_id
WHERE TRUE
GROUP BY brew.name
ORDER BY 2 DESC
LIMIT 10
;"""
df = pd.read_sql_query(full_join_query, con=conn)
df

In [ ]:
# Q3
full_join_query = """SELECT beers.name, brew.name, abv
FROM beers
JOIN breweries as brew on brew.id = beers.brewery_id
WHERE TRUE
AND brew.country = 'France'
ORDER BY 3 DESC
LIMIT 10
;"""
df = pd.read_sql_query(full_join_query, con=conn)
df

In [ ]:
# Q4
full_join_query = """SELECT brew.country, count(*) as nb_porter, AVG(ABV) as abv_mean
FROM beers
JOIN breweries as brew on brew.id = beers.brewery_id
JOIN styles on styles.id = beers.style_id
WHERE TRUE
AND styles.style_name = 'Porter'
GROUP BY brew.country
ORDER BY 2 DESC
;"""
df = pd.read_sql_query(full_join_query, con=conn)
df

In [ ]:
# Q5 - observation : certaines opérations "simples" sont un peu compliquées à réaliser en SQL
q = """
WITH country_cnt AS (
    SELECT 
        brew.country AS country,
        COUNT(*) AS cnt
    FROM beers
    JOIN breweries AS brew ON brew.id = beers.brewery_id
    GROUP BY brew.country
    ORDER BY cnt DESC
), ranked_countries AS (
    SELECT
        country, cnt, ROW_NUMBER() OVER (ORDER BY cnt) as rnk
    FROM country_cnt
), nlines AS (
    SELECT count(*) as nn
    FROM country_cnt
), proxy_median AS (
    SELECT 
        country, cnt, POWER((rnk / nn) - 1/2, 2) as proxmed
    FROM ranked_countries
    LEFT JOIN nlines ON TRUE
)
SELECT * 
FROM proxy_median
ORDER BY proxmed ASC
LIMIT 1
;"""
df = pd.read_sql_query(q, con=conn)
df

# UC-2

# UC-3 search from a query

**Observation :** On ne pourra pas aller bien loin en terme de souplesse dans la requête

In [ ]:
QUERY = "stout"

q = f"""
WITH descriptions AS (
    SELECT 
        brew.name as brewery, beers.name as name, CONCAT(beers.descript, brew.descript) as descr
    FROM beers
    JOIN breweries as brew on brew.id = beers.brewery_id
    WHERE TRUE
    AND LENGTH(beers.descript) + LENGTH(beers.descript) > 2
)
SELECT *
FROM descriptions
WHERE True
AND descriptions.descr LIKE '%{QUERY}%'
;"""
pd.read_sql_query(q, con=conn)

# UC-4 vectorize items

In [ ]:
q = """
WITH data AS (
    SELECT 
        beers.id, beers.name, beers.abv, beers.ibu, beers.srm, beers.descript as beer_descr,
        brew.descript as brewer_descript, brew.name as brewery,
        styles.style_name
    FROM beers
    LEFT JOIN breweries as brew on brew.id = beers.brewery_id
    LEFT JOIN styles on styles.id = beers.style_id
), descriptions AS (
    SELECT 
        id,
        CONCAT('the beer ', name, ' from brewery ', brewery, ' (', brewer_descript, ') crafts the beer ', name, ' defined as ', beer_descr, '. Spec of the beer are: ABV=', abv, ', IBU=', ibu, ', SRM=', srm) as to_vectorize
    FROM data
)
SELECT 
    id, to_vectorize
FROM descriptions
WHERE True
    AND id % 12 = 3
;"""
df = pd.read_sql_query(q, con=conn)

In [ ]:
import requests
from typing import List
import numpy as np

class JinaEmbedder:
    
    URL = 'https://api.jina.ai/v1/embeddings'
    EMBEDDING_NAME = "jina-embeddings-v2-base-en"
    bearer_token = 'Bearer jina_85ba1ab9e5ff4017b3d216ebb8734f27xzJ9WyoYBFwqks9lOaNLHryw_Yyz'

    @staticmethod
    def http_json_to_vec(http_json: dict):
        return np.array(
            [
                sentence["embedding"]
                for sentence in http_json["data"]
            ]
        )        

    @classmethod
    def embed(cls, str_to_vectorize: List[str] | str) -> dict:
        if isinstance(str_to_vectorize, str):
            str_to_vectorize = [str_to_vectorize]
        headers = {
            'Content-Type': 'application/json',
            'Authorization': cls.bearer_token
        }
        data = {
            'model': cls.EMBEDDING_NAME,
            'normalized': True,
            'embedding_type': 'float',
            'input': str_to_vectorize
        }
        
        response = requests.post(cls.URL, headers=headers, json=data)

        if response.status_code != 200:
            raise ValueError(f"Error code {response.status_code} on this call")

        return JinaEmbedder.http_json_to_vec(response.json())


In [ ]:
my_embeddings = JinaEmbedder.embed(df.iloc[:, 1].to_list())